In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from tqdm.notebook import tqdm
import matplotlib as plt

import sys
sys.path.append('..')

from src.datasets import load_dataset
from src.evaluation.evaluation import downvote_seen_items, topn_recommendations, model_evaluate

import json
import torch
from src.models import NGCF

In [ ]:
data_path = '../data'

training_yelp, testset_yelp, holdout_yelp, training_matrix_yelp, data_description_yelp = load_dataset('yelp', data_path)
training_movielens, testset_movielens, holdout_movielens, training_matrix_movielens, data_description_movielens = load_dataset('movielens', data_path)

In [ ]:
with open('../configs/ngcf_base.json') as f:
    config = json.load(f)

n_users, n_items = training_matrix_movielens.shape
config['model_args']['n_users'] = n_users
config['model_args']['n_items'] = n_items

model = NGCF(**config['model_args'])
model.load_state_dict(torch.load('../lightning_logs/version_30/checkpoints/epoch=399-step=2399.ckpt')['state_dict'])
model.eval();

In [ ]:
import torch
from torch_scatter import scatter_sum


class ALS:
    def __init__(self, factors, iterations=100, regularization=0.1, device='cpu', callback=None):
        
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.device = device
        self.callback = callback
        
    def fit(self, R):

        m, n = R.shape
        
        P = torch.randn(m, self.factors, device=self.device) / np.sqrt(self.factors)
        Q = torch.randn(n, self.factors, device=self.device) / np.sqrt(self.factors)
        I = torch.eye(self.factors, device=self.device)
        
        R = R.tocoo()
        rows, cols = torch.from_numpy(np.stack(R.nonzero())).to(torch.long)
        
        rows = rows.to(self.device)
        cols = cols.to(self.device)
        
        logs = []
        for _ in tqdm(range(self.iterations)):
                
            A = torch.linalg.inv(Q.T @ Q + self.regularization * I) @ Q.T
            P = scatter_sum(A[:, cols], rows).T

            A = torch.linalg.inv(P.T @ P + self.regularization * I) @ P.T
            Q = scatter_sum(A[:, rows], cols).T
            
            if self.callback is not None:
                log = self.callback(P, Q)
                logs.append(log)
                
        self.P = P
        self.Q = Q
        
        return np.array(logs)
    
    def predict(self, id_user, k=None):
        
        if k is None:
            
            k = len(self.Q)
            
        p = self.P[id_user]
        
        scores = self.Q @ p
        
        top_k = torch.argsort(scores, descending=True)[:k]
        
        return top_k
    
    @torch.no_grad()
    def score_users(self, user_ids):
        user_ids = torch.from_numpy(user_ids).to(torch.long).to(self.device)
        scores = self.P[user_ids] @ self.Q.T
        return scores.cpu().numpy()


class eALS:
    def __init__(self, factors, iterations=100, w=1, c=1, regularization=0.1, device='cpu', callback=None):
        self.w = w
        self.c = c
        self.factors = factors
        self.iterations = iterations
        self.regularization = regularization
        self.device = device
        self.callback = callback
        
    def fit(self, X):
        K = self.factors
        M, N = X.shape

        self.P = torch.randn(M, K, device=self.device) / np.sqrt(K)
        self.Q = torch.randn(N, K, device=self.device) / np.sqrt(K)
        
        X = X.tocoo()
        rows, cols = torch.from_numpy(np.stack(X.nonzero())).to(torch.long)
        
        rows = rows.to(self.device)
        cols = cols.to(self.device)
        
        c = self.c
        if isinstance(c, int):
            c = torch.ones(N, device=self.device) * c
        else:
            c = torch.tensor(c, device=self.device)
        
        R_hat = (self.P[rows] * self.Q[cols]).sum(axis=1)
        
        logs = []
        
        for _ in tqdm(range(self.iterations)):
            S_q = (c.unsqueeze(1) * self.Q).T @ self.Q

            for f in range(K):
                r_hat = R_hat - self.P[rows, f] * self.Q[cols, f]

                nominator = scatter_sum((self.w - (self.w - c[cols]) * r_hat) * self.Q[cols, f], rows)
                nominator -= self.P @ S_q[:, f] - self.P[:, f] * S_q[f, f]

                denominator = scatter_sum((self.w - c[cols]) * self.Q[cols, f] ** 2, rows) + S_q[f, f]
                self.P[:, f] = nominator / (denominator + self.regularization)

                R_hat = r_hat + self.P[rows, f] * self.Q[cols, f]

            S_p = self.P.T @ self.P
            for f in range(K):
                r_hat = R_hat - self.P[rows, f] * self.Q[cols, f] 

                nominator = scatter_sum((self.w - (self.w - c[cols]) * r_hat) * self.P[rows, f], cols)
                nominator -= c * (self.Q @ S_p[:, f] - self.Q[:, f] * S_p[f, f])

                denominator = scatter_sum((self.w - c[cols]) * self.P[rows, f] ** 2, cols) + c * S_p[f, f] 
                self.Q[:, f] = nominator / (denominator + self.regularization)

                R_hat = r_hat + self.P[rows, f] * self.Q[cols, f]
                
            if self.callback is not None:
                log = self.callback(self.P, self.Q)
                logs.append(log)
            
        return np.array(logs)
    
    def predict(self, id_user, k=None):
        
        if k is None:
            
            k = len(self.Q)
            
        p = self.P[id_user]
        
        scores = self.Q @ p
        
        top_k = torch.argsort(scores, descending=True)[:k]
        
        return top_k
    
    @torch.no_grad()
    def score_users(self, user_ids):
        user_ids = torch.from_numpy(user_ids).to(torch.long).to(self.device)
        scores = self.P[user_ids] @ self.Q.T
        return scores.cpu().numpy()

In [62]:
model = ALS(factors=100, device='cuda')
model.fit(training_matrix_yelp);

  0%|          | 0/100 [00:00<?, ?it/s]

In [54]:
# !!! scores_X - результат работы модели X, т.е. numpy.ndarray шейпа: scores_X.shape=(число юзеров в холдауте, число всех айтемов),
# где элементы - предсказанные скоры айтемов для каждого юзера (не индексы айтемов, а именно скоры).
scores_yelp = model.score_users(holdout_yelp.userid.values)
# scores_movielens = model.score_users(holdout_movielens.userid.values)

In [55]:
downvote_seen_items(scores_yelp, testset_yelp, data_description_yelp)
# downvote_seen_items(scores_movielens, testset_movielens, data_description_movielens)

In [56]:
topn = 10

recs_yelp = topn_recommendations(scores_yelp, topn=topn)
# recs_movielens = topn_recommendations(scores_movielens, topn=topn)

In [60]:
# import implicit
# als = implicit.als.AlternatingLeastSquares()
# als.fit(training_matrix_movielens)
# recs_movielens = als.recommend_all(training_matrix_movielens, filter_already_liked_items=True, N=topn)

C:\Users\Константин\AppData\Roaming\Python\Python38\site-packages\implicit\utils.py:28: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [58]:
datasets = ['Yelp: ', 'MovieLens: ']

In [59]:
hr, mrr, ndcg, cov = model_evaluate(recs_yelp, holdout_yelp, data_description_yelp, topn=topn)
print(datasets[0])
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')
print(f'nDCG@{topn} = {ndcg:.4f}')
print(f'COV@{topn} = {cov:.4f}')

Yelp: 
HR@10 = 0.0418
MRR@10 = 0.0155
nDCG@10 = 0.0216
COV@10 = 0.0625


In [45]:
hr, mrr, ndcg, cov = model_evaluate(recs_movielens, holdout_movielens, data_description_movielens, topn=topn) 
print(datasets[1])
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')
print(f'nDCG@{topn} = {ndcg:.4f}')
print(f'COV@{topn} = {cov:.4f}')

MovieLens: 
HR@10 = 0.0990
MRR@10 = 0.0359
nDCG@10 = 0.0505
COV@10 = 0.3653
